Instalo a biblioteca opencv-python para poder dar `import cv2`

In [ ]:
!pip install opencv-python
!pip install scikit-image
!pip install imagehash
!pip install -U sentence-transformers
!pip install rdflib

# Imports

In [ ]:
import requests
import cv2
from PIL import Image
import imagehash
import rdflib
from rdflib import Graph
import os
import shutil

# Variáveis globais para nomes de diretórios

In [ ]:
name_dir_RDF_files_original = 'RDF'
name_dir_RDF_files_mod = 'RDF_mod'
name_dir_imgs = 'Imagens'

# Lendo os arquivos RDF e salvando as imagens a partir das suas URIs

Uso `os.listdir('RDF')` para pegar os nomes dos arquivos do diretório RDF, que contém os arquivos RDF com os dados sobre as imagens.

Com rdflib.Graph(), posso retornar as triplas

Por algum motivo, o código dá erro se os arquivos RDF usarem `@PREFIX dbo: <http://dbpedia.org/ontology/>` . ou `@PREFIX dbp: <http://dbpedia.org/property/> .`, então optei por não usar nos arquivos

In [ ]:
# A funcao recebe como parametros:
# - Uma lista com os paths dos arquivos RDF a serem lidos
# - O nome do diretorio onde as imagens serao salvas
# - Um dicionario que mapeia a URI onde cada imagem foi baixada a seu arquivo
# - Um dicionario que mapeia o path do arquivo RDF de cada imagem
def le_RDF_salva_imgs(list_path_RDF_files, name_dir_imgs, dict_img_URI, dict_img_RDF_path):
    img_counter = 1 # contador usado para nomear as imagens baixadas a partir da URI retirada do arquivo RDF

    # Se o diretorio onde as imagens serao salvas nao existe, crio ele
    if not os.path.exists('Imagens'):
            os.makedirs('Imagens')

    for filepath in list_path_RDF_files: # pego todos os arquivos do diretorio
        # Grafo que contem a tripla (obra, rdf:resource, URI)
        graph_RDF_file = Graph() # o grafo mantinha as triplas anteriores, entao deve ser resetado
        graph_RDF_file.parse(filepath, format="ttl")
        path_img = name_dir_imgs + '\img_'+"%04d" % (img_counter,)+'.jpg'
        dict_img_RDF_path[path_img] = filepath # pego o arquivo RDF referente a imagem
        img_counter += 1    
        for subj, pred, obj in graph_RDF_file:
            # Se o predicado for rdf:resource, pego o objeto, que sera a URL da imagem, e realizo o download desta
            if str(pred) == 'http://www.w3.org/1999/02/22-rdf-syntax-ns#resource':
                img_data = requests.get(str(obj)).content
                with open(path_img, 'wb') as handler:
                    handler.write(img_data)
                dict_img_URI[path_img] = str(obj) # salvo o par (path_img, URI) no dicionario

In [ ]:
import os

##desnecessario
dict_img_URI = {} # dicionario que tem o caminho para o arquivo da imagens como chave, com sua URI como valor (acabei nao usando)

# Dicionario que mapeia o path do arquivo RDF de cada imagem
dict_img_RDF_path = {}

# Pego o nome dos arquivos RDF originais, que usarei para obter o path deles
list_RDF_files_original_name = os.listdir(name_dir_RDF_files_original) # pego todos os arquivos RDF do diretório

#print(list_RDF_files_original_name) ##Debug

# Pego o path para os arquivos RDF originais, que usarei para salvar as imagens do arquivo RDF
list_RDF_files_original_path = [name_dir_RDF_files_original + '\\' + file_name for file_name in list_RDF_files_original_name]

#print(list_RDF_files_original_path) ##Debug

In [ ]:
le_RDF_salva_imgs(list_RDF_files_original_path, name_dir_imgs, dict_img_URI, dict_img_RDF_path)

# Alterando os arquivos RDF

Devido a dificuldades para encontrar uma maneira boa e consistente de comparar imagens, para fins de demonstração, irei criar um dicionário para separar as imagens semelhantes

In [ ]:
# Dicionario no formato {img_path : sim_grp}
# Onde sim_grp denota o grupo de semelhanca de uma imagem
# Imagens semelhantes pertencem ao mesmo grupo
dict_img_sim_grp = {}

# GRUPO 10001 - Moca com o Brinco de Perola
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0001' + '.jpg'] = 10001
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0002' + '.jpg'] = 10001
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0003' + '.jpg'] = 10001

# GRUPO 10002 - Mona Lisa
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0004' + '.jpg'] = 10002
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0005' + '.jpg'] = 10002
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0006' + '.jpg'] = 10002
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0007' + '.jpg'] = 10002

# GRUPO 10003 - O Nascimento de Venus
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0008' + '.jpg'] = 10003
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0009' + '.jpg'] = 10003
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0010' + '.jpg'] = 10003

# GRUPO 10004 - A Noite Estrelada
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0011' + '.jpg'] = 10004
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0012' + '.jpg'] = 10004
dict_img_sim_grp[name_dir_imgs + '\\' + 'img_0013' + '.jpg'] = 10004

Agora, para cada grupo, vou iterar sobre todos os seus arquivos, e adicionar as triplas `(subj,pred,obj)` no `dicionario dict_grp_triples`

In [ ]:
# A funcao recebe como parametros:
# - Um dicionario que mapeia o path do arquivo RDF de cada imagem
# - Um dicionario que mapeia o grupo de semelhanca de cada imagem
# - Um dicionario que mapeia uma lista de triplas de cada arquivo RDF referente a uma imagem
def le_RDF_armazena_tripla_no_dict(dict_img_RDF_path, dict_img_sim_grp, dict_grp_triples):
    #print(list_path_RDF_files) ##Debug

    for img_path in dict_img_sim_grp: # pego todos os arquivos do diretorio
        #print(img_path) ##Debug

        # Grafo que contem a tripla (SUBJ, PRED, OBJ)
        graph_RDF_file = Graph() # o grafo mantem as triplas anteriores, entao deve ser resetado em cada iteracao

        # Pego o numero do grupo de semelhanca
        grp_num = dict_img_sim_grp[img_path]

        # Pego o path do arquivo RDF referente a imagem, para poder abri-lo
        rdf_path = dict_img_RDF_path[img_path]

        graph_RDF_file.parse(rdf_path, format="ttl")
        for subj, pred, obj in graph_RDF_file:
            # Se o predicado nao for rdf:resource, adiciono a tripla no dicionario
            if str(pred) != 'http://www.w3.org/1999/02/22-rdf-syntax-ns#resource':
                #print('SUBJ:',subj, '\nPRED:',pred, '\nOBJ:',obj,'\n==========') ##Debug
                triple = (str(subj), str(pred), str(obj))
                # Se ainda nao atribui triplas para o grupo, nao posso dar append, entao crio uma lista vazia
                if grp_num not in dict_grp_triples:
                    dict_grp_triples[grp_num] = []
                if triple not in dict_grp_triples[grp_num]:
                    dict_grp_triples[grp_num].append((str(subj), str(pred), str(obj)))

In [ ]:
# Dicionario no formato {sim_grp : [(subj,pred,obj)]}
# Cada sim_grp tem uma lista de triplas presentes nos arquivos RDF pertencentes a um sim_grp
# Imagens semelhantes pertencem ao mesmo grupo
dict_grp_triples = {}

le_RDF_armazena_tripla_no_dict(dict_img_RDF_path, dict_img_sim_grp, dict_grp_triples)

In [ ]:
for sim_grp in dict_grp_triples:
    print("Grupo:", sim_grp)
    for triple in dict_grp_triples[sim_grp]:
        print(triple)

# Leio os arquivos RDF e adiciono triplas que não estão presentes neles

In [ ]:
import os
import shutil

# A funcao recebe como parametros:
# - Um dicionario que mapeia o path do arquivo RDF de cada imagem
# - Um dicionario que mapeia o grupo de semelhanca de cada imagem
# - Um dicionario que mapeia uma lista de triplas de cada arquivo RDF referente a uma imagem
# - O nome do diretorio onde os arquivos modificados serao salvos
def le_RDF_salva_copia_modificada(dict_img_RDF_path, dict_img_sim_grp, dict_grp_triples, name_dir_RDF_files_mod):
    list_RDF_file_triples = [] # triplas que ja estao no arquivo RDF, uso pra comparar com as triplas do dicionario
    list_triples_add = [] # triplas que irei adicionar no arquivo RDF
    
    # Se o diretorio onde os arquivos modificados serao salvos nao existe, crio ele
    if not os.path.exists(name_dir_RDF_files_mod):
            os.makedirs(name_dir_RDF_files_mod)

    for img_path in dict_img_sim_grp: # pego todos os arquivos do diretorio
        list_RDF_file_triples = [] # novo arquivo, reseto a lista
        list_triples_add = [] # novo arquivo, reseto a lista

        # Grafo que contem a tripla (SUBJ, PRED, OBJ)
        graph_RDF_file = Graph() # o grafo mantem as triplas anteriores, entao deve ser resetado em cada iteracao

        # Pego o numero do grupo de semelhanca
        sim_grp = dict_img_sim_grp[img_path]

        # Pego o path do arquivo RDF referente a imagem, para poder abri-lo
        rdf_path = dict_img_RDF_path[img_path]

        graph_RDF_file.parse(rdf_path, format="ttl")

        for subj, pred, obj in graph_RDF_file:
            trpl = (str(subj), str(pred), str(obj))
            list_RDF_file_triples.append(trpl)

        # Para cada tripla do dicionario, irei verificar se ela esta no arquivo RDF
        for triple in dict_grp_triples[sim_grp]:
            if triple not in list_RDF_file_triples:
                list_triples_add.append(triple)

        new_rdf_file = name_dir_RDF_files_mod + '\\' + rdf_path[4::]
        # Crio uma copia do arquivo no diretorio de arquivos modificados
        shutil.copy(rdf_path, new_rdf_file)

        with open(new_rdf_file, 'a') as new_rdf_file:
            for triple in list_triples_add:
                if 'http' in triple[2]:
                    line = '<' + triple[0] + '>' + " " + '<' + triple[1] + '>' + " " + '<' + triple[2] + '>' + " ."
                else:
                    line = '<' + triple[0] + '>' + " " + '<' + triple[1] + '>' + " " + '\"' + triple[2] + '\"' + " ."
                new_rdf_file.write('\n' + line + '\n')
            

In [ ]:
le_RDF_salva_copia_modificada(dict_img_RDF_path, dict_img_sim_grp, dict_grp_triples, name_dir_RDF_files_mod)

# Plotando o grafo
Plotando o grafo

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

def le_RDF_plota_grafo(rdf_path):
    graph_RDF_file = Graph() # grafo que contem a tripla (obra, rdf:resource, URI)
    result = graph_RDF_file.parse(rdf_path, format="ttl")

    graph = rdflib_to_networkx_multidigraph(result)

    pos = nx.spring_layout(graph, scale=2)
    edge_labels = nx.get_edge_attributes(graph, 'r')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
    nx.draw(graph, with_labels=True)

    #if not in interactive mode for 
    plt.show()

In [ ]:
path = name_dir_RDF_files_mod + "\\img_MonaLisa_0001.ttl"

le_RDF_plota_grafo(path)